In [ ]:
import pandas as pd
import json
import requests
import time
import threading
import pyodbc
from concurrent.futures import ThreadPoolExecutor
import ast
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import os
from datetime import datetime, date

'''
ask token
'''    
url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
payload = {
    "grant_type": "password",
    "client_id": "351fb11916fc85504ba5f6ec3b96988b",
    "client_secret": "8f9355878504005ba782ca049d013d4e",
    "redirect_uri": "https://crm-p10.xiaoshouyi.com",
    "username": "11020964@twkd.com",
    "password": "Kd0627ubGHagpQ"
}

response = requests.post(url, data=payload)
content = response.json()
ac_token = content["access_token"]


'''
obtain table type to get the entityType
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/xobjects/customEntity25__c/busiType"
headers = {
    "Authorization": f"Bearer {ac_token}"
    #"Content-Type":"application/json"    #json data type
}
response2 = requests.get(url_2, headers = headers)
content2 = response2.json()
print(content2)
entityType_df = pd.json_normalize(content2["data"]["records"]) 


'''
昨日台灣最大值
'''
date_0 = (datetime.now() - timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
date_0_scrm = int(date_0.timestamp() * 1000)

date_00 = (datetime.now() - timedelta(days=1)).replace(hour=23, minute=59, second=59, microsecond=999999)
date_00_scrm = int(date_00.timestamp() * 1000)


url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
Apply_TW_Yesterday= pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''SELECT 
                    name 
                    ,customItem150__c 營運單位審核寄送日期時間 
                    ,customItem117__c 退件原因分類
                    ,customItem63__c  營運單位審核_是否提供型錄
                    ,entityType
                    ,createdAt  
                    ,customItem156__c as 嘉義撿貨流水編號
                    ,customItem157__c as 嘉義審核時間
                    ,customItem97__c  as 申請物品
                    FROM customEntity25__c
                    WHERE customItem150__c >= {date_0_scrm} 
                    AND customItem150__c <= {date_00_scrm} 
                   and entityType = '2906694858215364'
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Apply_TW_Yesterday = pd.concat([Apply_TW_Yesterday, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
Apply_TW_Yesterday['營運單位審核寄送日期時間'] = Apply_TW_Yesterday['營運單位審核寄送日期時間'].replace('', np.nan)
Apply_TW_Yesterday['營運單位審核寄送日期時間'] = Apply_TW_Yesterday['營運單位審核寄送日期時間'].astype(float)
Apply_TW_Yesterday['營運單位審核寄送日期時間'] = Apply_TW_Yesterday['營運單位審核寄送日期時間'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
Apply_TW_Yesterday['營運單位審核寄送日期時間'] = Apply_TW_Yesterday['營運單位審核寄送日期時間'].dt.tz_convert('Asia/Taipei')
Apply_TW_Yesterday['營運單位審核寄送日期時間'] = Apply_TW_Yesterday['營運單位審核寄送日期時間'].dt.strftime('%Y-%m-%d %H:%M:%S')

Apply_TW_Yesterday['createdAt'] = Apply_TW_Yesterday['createdAt'].replace('', np.nan)
Apply_TW_Yesterday['createdAt'] = Apply_TW_Yesterday['createdAt'].astype(float)
Apply_TW_Yesterday['createdAt'] = Apply_TW_Yesterday['createdAt'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
Apply_TW_Yesterday['createdAt'] = Apply_TW_Yesterday['createdAt'].dt.tz_convert('Asia/Taipei')
Apply_TW_Yesterday['createdAt'] = Apply_TW_Yesterday['createdAt'].dt.strftime('%Y-%m-%d %H:%M:%S')


Apply_TW_Yesterday['營運單位審核寄送日期時間'] = pd.to_datetime(Apply_TW_Yesterday['營運單位審核寄送日期時間'])
# 取得昨天的日期
yesterday = (datetime.now() - timedelta(days=1)).date()

# 篩選昨天 & 16:00:59 前的數據
Apply_TW_Yesterday = Apply_TW_Yesterday[
    (Apply_TW_Yesterday['營運單位審核寄送日期時間'].dt.date == yesterday) &  # 限定昨天的數據
    (Apply_TW_Yesterday['營運單位審核寄送日期時間'].dt.time < pd.to_datetime('16:00:59').time())  # 時間篩選
]

TW_max_yesterday = Apply_TW_Yesterday['嘉義撿貨流水編號'].max()

# 提取最後三個字元
TW_max_yesterday_last_three_digits = TW_max_yesterday[-3:]
twno = 'TW' + date_0.strftime('%y%m%d') + "-"
print(f'台灣昨天日最大值：{twno} {TW_max_yesterday_last_three_digits}')
'''
台灣發放申請(customEntity25__c)
'''

url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
Apply_TW= pd.DataFrame()
 
while True:
    data = {
        "xoql": f'''SELECT 
                    id
                    ,name 
                    ,customItem150__c 營運單位審核寄送日期時間 
                    ,customItem117__c 退件原因分類
                    ,customItem63__c  營運單位審核_是否提供型錄
                    ,entityType
                    ,createdAt  
                    ,customItem156__c as 嘉義撿貨流水編號
                    ,customItem157__c as 嘉義審核時間
                    ,customItem97__c  as 申請物品
                    FROM customEntity25__c
                    where 
                   customItem150__c >= {date_0_scrm}
                   and entityType = '2906694858215364'
                   and customItem150__c IS NOT NULL
                   and customItem117__c IS NULL
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    Apply_TW = pd.concat([Apply_TW, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']
    
today = datetime.now().date()

Apply_TW['營運單位審核寄送日期時間'] = Apply_TW['營運單位審核寄送日期時間'].replace('', np.nan)
Apply_TW['營運單位審核寄送日期時間'] = Apply_TW['營運單位審核寄送日期時間'].astype(float)
Apply_TW['營運單位審核寄送日期時間'] = Apply_TW['營運單位審核寄送日期時間'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
Apply_TW['營運單位審核寄送日期時間'] = Apply_TW['營運單位審核寄送日期時間'].dt.tz_convert('Asia/Taipei')
Apply_TW['營運單位審核寄送日期時間'] = Apply_TW['營運單位審核寄送日期時間'].dt.strftime('%Y-%m-%d %H:%M:%S')

Apply_TW['createdAt'] = Apply_TW['createdAt'].replace('', np.nan)
Apply_TW['createdAt'] = Apply_TW['createdAt'].astype(float)
Apply_TW['createdAt'] = Apply_TW['createdAt'].apply(lambda x: pd.to_datetime(x / 1000.0, unit='s', utc=True))
Apply_TW['createdAt'] = Apply_TW['createdAt'].dt.tz_convert('Asia/Taipei')
Apply_TW['createdAt'] = Apply_TW['createdAt'].dt.strftime('%Y-%m-%d %H:%M:%S')


# 篩選今天 & 16:00:59 前的數據
Apply_TW['營運單位審核寄送日期時間'] = pd.to_datetime(Apply_TW['營運單位審核寄送日期時間'])


Apply_TW_today = Apply_TW[
    (Apply_TW['營運單位審核寄送日期時間'].dt.date == today) &  # 限定昨天的數據
    (Apply_TW['營運單位審核寄送日期時間'].dt.time < pd.to_datetime('16:00:59').time())  # 時間篩選
]

TW_max_today = Apply_TW_today['嘉義撿貨流水編號'].max()

# 提取最後三個字元
TW_max_today_last_three_digits = TW_max_today[-3:]
twno = 'TW' + date_0.strftime('%y%m%d') + "-"
print(f'台灣今日最大值：{twno} {TW_max_today_last_three_digits}')

Apply_TW = Apply_TW[Apply_TW['嘉義撿貨流水編號']==""]
Apply_TW['營運單位審核_是否提供型錄'] = Apply_TW['營運單位審核_是否提供型錄'].str.get(0)
Apply_TW_model = Apply_TW[~Apply_TW['申請物品'].str.contains('型錄|門框', case=False, na=False)]
Apply_TW_catalog = Apply_TW[Apply_TW['申請物品'].str.contains('型錄|門框', case=False, na=False)]
Apply_TW_catalog = Apply_TW_catalog[Apply_TW_catalog['營運單位審核_是否提供型錄'] == '是']

final_tw = pd.concat([Apply_TW_catalog, Apply_TW_model], ignore_index=True)


final_tw['營運單位審核寄送日期時間'] = pd.to_datetime(final_tw['營運單位審核寄送日期時間'])

twno_yesterday = 'TW' + yesterday.strftime('%y%m%d') + "-"  # TW250320-
twno_today = 'TW' + today.strftime('%y%m%d') + "-"  # TW250321-
yesterday_counter = int(TW_max_yesterday_last_three_digits) +1  # 16:00前
today_counter = int(TW_max_today_last_three_digits) +1 # 16:00前


# 逐行判斷
for i, row in final_tw.iterrows():
    date_part = row['營運單位審核寄送日期時間'].date()
    time_part = row['營運單位審核寄送日期時間'].time()

    if date_part == yesterday and time_part < datetime.strptime("16:00:00", "%H:%M:%S").time():
        # 昨天 16:00 前，繼續昨天的流水號
        final_tw.at[i, '嘉義撿貨流水編號'] = f"{twno_yesterday}{yesterday_counter:03d}"
        yesterday_counter += 1
    else:
        # 昨天 16:00 後 或 今天，使用新的日期流水號
        final_tw.at[i, '嘉義撿貨流水編號'] = f"{twno_today}{today_counter:03d}"
        today_counter += 1

# 顯示結果
print(final_tw)
# 轉換時間格式
final_tw['營運單位審核寄送日期時間'] = pd.to_datetime(final_tw['營運單位審核寄送日期時間'])
final_tw['嘉義審核時間'] = final_tw['營運單位審核寄送日期時間'].dt.strftime('%H:%M')


final_tw =final_tw[['id','嘉義撿貨流水編號','嘉義審核時間']]
final_tw.rename(columns={'嘉義撿貨流水編號':'customItem156__c','嘉義審核時間':'customItem157__c'} , inplace=True)
# final_tw['customItem157__c'] = str(final_tw['customItem157__c'] )

print(final_tw.dtypes)

'''
ask bulk id
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/bulk/v2/job"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/json"
}
data = {
      "data": {
        "operation": "update",####################################################################3
        "object": "customEntity25__c",
        "execOption": ["CHECK_RULE", "CHECK_DUPLICATE"]
    }
    
}
response = requests.post(url_2, headers = headers, json = data)
test = response.json()
bulk_id = test["result"]["id"]
print(bulk_id)


batch_size = 5000

# Calculate the number of batches
num_batches = (len(final_tw) + batch_size - 1) // batch_size
# Initialize a list to store responses
all_responses = []
# Initialize an index to keep track of the current batch
current_batch_index = 0

# While there are more batches to process
while current_batch_index < num_batches:
    start_index = current_batch_index * batch_size
    end_index = (current_batch_index + 1) * batch_size
    current_batch_df = final_tw.iloc[start_index:end_index]
    json_data = current_batch_df.to_dict(orient='records')
    
    url_2 = "https://api-p10.xiaoshouyi.com/rest/bulk/v2/batch"
    headers = {
        "Authorization": f"Bearer {ac_token}",
        "Content-Type":"application/json"
    }
    data = {
        "data": {
            "jobId": f"{bulk_id}",
            "datas": json_data
        }
    }
    response = requests.post(url_2, headers=headers, json=data)
    all_responses.append(response)
    current_batch_index += 1


